In [29]:
import tweepy
import pandas as pd
import yfinance as yf
import openai
import big_baller_moves
import json
import requests
from webdriver_manager.chrome import ChromeDriverManager
import time

In [30]:
#Put your Bearer Token in the parenthesis below
client1 = tweepy.Client(bearer_token=big_baller_moves.bossman_tingz["twitter_api_key"])

# Authenticate
auth = tweepy.OAuth1UserHandler(big_baller_moves.bossman_tingz["consumer_key"], big_baller_moves.bossman_tingz["consumer_secret"], big_baller_moves.bossman_tingz["access_token"], 
                                big_baller_moves.bossman_tingz["access_token_secret"])

# Create API object
api = tweepy.API(auth)

# openai key
openai.api_key = big_baller_moves.bossman_tingz["openai_api_key"]

In [31]:
#enter username to get the user ID
username = "Heisenberg_100k"

try:
    user = client1.get_user(username=username)
    user_id = user_id = user.data.id
    print("User ID for", username, ":", user_id)
except Exception as e:
    print("Error:", e)

User ID for Heisenberg_100k : 1750349614786691072


In [33]:
#get btc price (not working atm)

# Choose Token and Time
timestamp_open = pd.Timestamp('2024-05-02 05:09:00', tz='UTC')
unix_timestamp = int(timestamp_open.timestamp())
unix_timestamp_24 = unix_timestamp + (24*60*60)
unix_timestamp_48 = unix_timestamp + (48*60*60)
ckey = f"https://api.coingecko.com/api/v3/coins/bitcoin/history?date={unix_timestamp}&localization=false"
url = f"https://api.kraken.com/0/public/OHLC?pair=XBTUSD&interval=1&since={unix_timestamp}"
ckey24 = f"https://api.coingecko.com/api/v3/coins/bitcoin/history?date={unix_timestamp_24}&localization=false"
ckey48 = f"https://api.coingecko.com/api/v3/coins/bitcoin/history?date={unix_timestamp_48}&localization=false"

try:
        # Convert timestamp to Unix timestamp (Kraken API requires timestamp in seconds)
        unix_timestamp = int(timestamp.timestamp())

        # Kraken API endpoint for OHLC data
        url = f"https://api.kraken.com/0/public/OHLC?pair=XBTUSD&interval=1&since={unix_timestamp}"

response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            if 'result' in data and 'XBTUSD' in data['result'] and data['result']['XBTUSD']:
                # The response contains a list of lists, where each inner list represents OHLC data for a specific interval
                # We extract the closing price from the last list (most recent data point)
                bitcoin_price = float(data['result']['XBTUSD'][-1][4])
                return bitcoin_price
            else:
                print("No data available for the specified timestamp")
                return None
        else:
            print(f"Failed to retrieve data from Kraken API. Status code: {response.status_code}")
            return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None


IndentationError: unindent does not match any outer indentation level (<tokenize>, line 34)

In [ ]:
unix_timestamp

In [37]:
#get all tweets
tweets = client1.get_users_tweets(
    id= user_id,
    max_results=100,  # Number of tweets to retrieve (adjust as needed)
    tweet_fields=['id', 'text', 'created_at', 'entities', 'attachments'],  # Fields you want to retrieve for each tweet
    media_fields=['preview_image_url', 'url'],
    exclude = ['retweets','replies'],
    expansions=['attachments.media_keys', 'author_id']
)


In [38]:
# Process retrieved tweets

for tweet in tweets.data:
    print(f"Tweet ID: {tweet.id}, Text: {tweet.text}, Created At: {tweet.created_at}")


Tweet ID: 1790077422697693598, Text: Question:

Sell 1 naked GME call here.

Worth it?

Dangerous?

Only 1 shouldn’t be too bad?, Created At: 2024-05-13 17:51:30+00:00
Tweet ID: 1790077076105498901, Text: $PARA Open

See how Sony plays the buyout process.

Small. https://t.co/CceV23s8gb, Created At: 2024-05-13 17:50:07+00:00
Tweet ID: 1790026595027005823, Text: Sold my $INTC calls too soon. But shares doing well.

Anyone hopped in the INTC trade?, Created At: 2024-05-13 14:29:31+00:00
Tweet ID: 1790014939173630180, Text: $VKTX Open https://t.co/2G1TFQ8iaV, Created At: 2024-05-13 13:43:13+00:00
Tweet ID: 1790014760244621631, Text: $INTC Closed, switched to shares

In 1.15 (May 8th)

Out 1.29

+12.1% 
+$140

Switched to 400 shares instead. Still looking for 32-33’s. https://t.co/LeXt1fPshM, Created At: 2024-05-13 13:42:30+00:00
Tweet ID: 1789972227565273090, Text: $VKTX and $SMMT back on watch., Created At: 2024-05-13 10:53:29+00:00
Tweet ID: 1789968454356779401, Text: $GME selling call 

In [39]:
tweets

Response(data=[<Tweet id=1790077422697693598 text='Question:\n\nSell 1 naked GME call here.\n\nWorth it?\n\nDangerous?\n\nOnly 1 shouldn’t be too bad?'>, <Tweet id=1790077076105498901 text='$PARA Open\n\nSee how Sony plays the buyout process.\n\nSmall. https://t.co/CceV23s8gb'>, <Tweet id=1790026595027005823 text='Sold my $INTC calls too soon. But shares doing well.\n\nAnyone hopped in the INTC trade?'>, <Tweet id=1790014939173630180 text='$VKTX Open https://t.co/2G1TFQ8iaV'>, <Tweet id=1790014760244621631 text='$INTC Closed, switched to shares\n\nIn 1.15 (May 8th)\n\nOut 1.29\n\n+12.1% \n+$140\n\nSwitched to 400 shares instead. Still looking for 32-33’s. https://t.co/LeXt1fPshM'>, <Tweet id=1789972227565273090 text='$VKTX and $SMMT back on watch.'>, <Tweet id=1789968454356779401 text='$GME selling call spreads on watch.'>, <Tweet id=1789302719662158311 text='You know it is true. https://t.co/HRF5z5GmYK'>, <Tweet id=1789299906559262797 text='The volatility of volatility $VVIX is essent

In [40]:
def get_display_url(entities):
    if isinstance(entities, dict) and "urls" in entities:
        urls = entities["urls"]
        display_urls = [url.get("display_url") for url in urls]
        return display_urls
    else:
        return 0

# Apply the function to count display_url
df["image_url"] = df["entities"].apply(get_display_url)

selected_columns = ['id', 'text', 'created_at', 'image_url']

# Creating a new DataFrame with selected columns
heisenberg_tweets = df[selected_columns].copy()

In [41]:
heisenberg_tweets

,id,text,created_at,image_url
0,1790077422697693598,Question:\n\nSell 1 naked GME call here.\n\nWorth it?\n\nDangerous?\n\nOnly 1 shouldn’t be too bad?,2024-05-13 17:51:30+00:00,0
1,1790077076105498901,$PARA Open\n\nSee how Sony plays the buyout process.\n\nSmall. https://t.co/CceV23s8gb,2024-05-13 17:50:07+00:00,[pic.twitter.com/CceV23s8gb]
2,1790026595027005823,Sold my $INTC calls too soon. But shares doing well.\n\nAnyone hopped in the INTC trade?,2024-05-13 14:29:31+00:00,0
3,1790014939173630180,$VKTX Open https://t.co/2G1TFQ8iaV,2024-05-13 13:43:13+00:00,[pic.twitter.com/2G1TFQ8iaV]
4,1790014760244621631,"$INTC Closed, switched to shares\n\nIn 1.15 (May 8th)\n\nOut 1.29\n\n+12.1% \n+$140\n\nSwitched to 400 shares instead. Still looking for 32-33’s. https://t.co/LeXt1fPshM",2024-05-13 13:42:30+00:00,"[pic.twitter.com/LeXt1fPshM, pic.twitter.com/LeXt1fPshM]"
...,...,...,...,...
93,1783854096040292837,$SOXL Closed\n\nIn 32.38 (April 19)\n\nOut 37.57\n\nGood enough https://t.co/fmM5DMrJan,2024-04-26 13:42:13+00:00,[pic.twitter.com/fmM5DMrJan]
94,1783543215695753574,$META 440. \n\nDo it for daddy.,2024-04-25 17:06:53+00:00,0
95,1783514573951365454,$META Need a GOOG and a MSFT beat to get META going tomorrow.\n\n🤞,2024-04-25 15:13:05+00:00,0
96,1783230723195052350,$META failed.\n\nWe gloat the winners.\n\nWe accept defeats on the losers.\n\nNeed a miracle by Friday. Otherwise -99%.,2024-04-24 20:25:09+00:00,0


In [42]:
df = pd.DataFrame(tweets.data)

In [43]:
#helper counting function to check display_urls

def count_display_url(entities):
    if isinstance(entities, dict) and "urls" in entities:
        urls = entities["urls"]
        display_urls = [url.get("display_url") for url in urls]
        print(display_urls)
        return sum(1 for url in display_urls if url is not None)
    else:
        return 0

# Apply the function to count display_url
count = df["entities"].apply(count_display_url).sum()

print("Number of rows with display_url:", count)

['pic.twitter.com/CceV23s8gb']
['pic.twitter.com/2G1TFQ8iaV']
['pic.twitter.com/LeXt1fPshM', 'pic.twitter.com/LeXt1fPshM']
['pic.twitter.com/HRF5z5GmYK']
['x.com/i/web/status/1…']
['x.com/i/web/status/1…']
['pic.twitter.com/FJwJ8S1RdE']
['x.com/i/web/status/1…', 'x.com/Heisenberg_100…']
['pic.twitter.com/shmUd4IV7w']
['pic.twitter.com/1FrUXsjpbc']
['pic.twitter.com/FsZa8r2C56']
['pic.twitter.com/WNy2WINVwq']
['pic.twitter.com/wZl8ISwGxb']
['pic.twitter.com/iigSxnAU8S']
['pic.twitter.com/qsWN4xdihq']
['pic.twitter.com/rL3Y94Ex9E']
['x.com/i/web/status/1…']
['x.com/i/web/status/1…', 'x.com/ShiKitYip1/sta…']
['pic.twitter.com/aIjR48ncY0']
['pic.twitter.com/64syimBVcM']
['pic.twitter.com/bMkPWxmKx4']
['x.com/i/web/status/1…']
['pic.twitter.com/9Wo5JfC8mO']
['pic.twitter.com/pm160UzPGD', 'pic.twitter.com/pm160UzPGD', 'pic.twitter.com/pm160UzPGD']
['pic.twitter.com/xtQMTvDNEY']
['pic.twitter.com/CcLC9F7DSF']
['pic.twitter.com/UproWGtGsH']
['pic.twitter.com/RFpL6lKTGj']
['pic.twitter.com/Wvnh

In [44]:
df[df['attachments'].isnull()]['entities']

0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

In [45]:
ticker_symbol = 'AAPL'

# Fetch historical data
stock_data = yf.Ticker(ticker_symbol)

# Get historical market data
historical_data = stock_data.history(period="max")

print(historical_data)

                                 Open        High         Low       Close  \
Date                                                                        
1980-12-12 00:00:00-05:00    0.099058    0.099488    0.099058    0.099058   
1980-12-15 00:00:00-05:00    0.094321    0.094321    0.093890    0.093890   
1980-12-16 00:00:00-05:00    0.087429    0.087429    0.086999    0.086999   
1980-12-17 00:00:00-05:00    0.089152    0.089582    0.089152    0.089152   
1980-12-18 00:00:00-05:00    0.091737    0.092167    0.091737    0.091737   
...                               ...         ...         ...         ...   
2024-05-07 00:00:00-04:00  183.201504  184.649537  181.074400  182.152924   
2024-05-08 00:00:00-04:00  182.602329  182.822032  181.204216  182.492477   
2024-05-09 00:00:00-04:00  182.312720  184.409882  181.863333  184.320007   
2024-05-10 00:00:00-04:00  184.899994  185.089996  182.130005  183.050003   
2024-05-13 00:00:00-04:00  185.434998  187.100006  184.619995  186.279999   

In [46]:
client = openai
df1 = df.copy().head(5)
pd.set_option('display.max_colwidth', None)

In [47]:
#for astronomer
system_msg = "Tell me if the message is suggesting opening a long, short or N/A and explain why. Some messages might indicate that they are shorting in the short-term but the long-term view indicates a bullish sentiment. In this case, we should tag the tweet as a short position. If the tweet indicates anything around taking profit or being in a short position call it a short."

In [48]:
#heisenberg
system_msg = "Tell me if the message is suggesting opening a long, short or N/A and explain why. Some messages might indicate that they are shorting in the short-term but the long-term view indicates a bullish sentiment. In this case, we should tag the tweet as a short position. If the tweet indicates anything around taking profit or being in a short position call it a short."

In [49]:
def get_response(text) :
    response = client.chat.completions.create(model="gpt-3.5-turbo", 
                                              messages=[ {"role":"system","content":"Tell me if the message is suggesting opening a long, short or N/A in one word. Some messages might indicate that they are shorting in the short-term but the long-term view indicates a bullish sentiment. In this case, we should tag the tweet as a short position. If the tweet indicates anything around taking profit or being in a short position call it a short." }, 
                                                        {"role":"user","content": text}])
   
    return response.choices[0].message.content.strip()



In [50]:
def get_response_guided(text) :
    response = client.chat.completions.create(model="gpt-3.5-turbo", 
                                              messages=[ {"role":"system","content":"Your job is to tag tweets as either opening a long position or short position or N/A. Provide a reason explaining why the tweet is a long or short or n/a." }, 
                                                        {"role":"user","content": "Taking profit"},
                                                        {"role":"assistant","content": "Short, the poster is taking profit."}])
   
    return response.choices[0].message.content.strip()


In [51]:
from bs4 import BeautifulSoup
from selenium import webdriver



def get_response_image(text) :
    if text == 0:
        return "No image available"
    if not isinstance(text, str):
        print(text)
        text = text[0]
    try:
        response = client.chat.completions.create(model="gpt-4o", 
                                                   messages=[
        {
          "role": "user",
          "content": [
            {"type": "text", "text": "What option play is this image describing?"},
            {
              "type": "image_url",
              "image_url": {
                "url": text,
              },
            },
          ],
        }
      ],
      max_tokens=300,)
   
        return response.choices[0].message.content.strip()
    except Exception as e:
        print(e)
        pass

In [52]:
df1['Direction (Guided)'] = df1['text'].apply(get_response_guided)
df1['Direction'] = df1['text'].apply(get_response)

In [53]:

heisenberg_tweets['Image'] = heisenberg_tweets['image_url'].apply(get_response_image)


['pic.twitter.com/CceV23s8gb']
Error code: 400 - {'error': {'message': 'Invalid image.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_image'}}
['pic.twitter.com/2G1TFQ8iaV']
Error code: 400 - {'error': {'message': 'Invalid image.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_image'}}
['pic.twitter.com/LeXt1fPshM', 'pic.twitter.com/LeXt1fPshM']
Error code: 400 - {'error': {'message': 'Invalid image.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_image'}}
['pic.twitter.com/HRF5z5GmYK']
Error code: 400 - {'error': {'message': 'Invalid image.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_image'}}
['x.com/i/web/status/1…']
Error code: 400 - {'error': {'message': 'Invalid image.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_image'}}
['x.com/i/web/status/1…']
Error code: 400 - {'error': {'message': 'Invalid image.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_image'}}
['

Error code: 400 - {'error': {'message': 'Invalid image.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_image'}}
['pic.twitter.com/y1dM3zFGzt']
Error code: 400 - {'error': {'message': 'Invalid image.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_image'}}
['x.com/i/web/status/1…', 'x.com/kirk_barnard/s…']
Error code: 400 - {'error': {'message': 'Invalid image.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_image'}}
['pic.twitter.com/RbpVmOVfeU']
Error code: 400 - {'error': {'message': 'Invalid image.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_image'}}
['x.com/i/web/status/1…']
Error code: 400 - {'error': {'message': 'Invalid image.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_image'}}
['pic.twitter.com/OjXQoj4O34']
Error code: 400 - {'error': {'message': 'Invalid image.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_image'}}
['x.com/Heisenberg_100…']
Error code: 

In [54]:

selected = ['text','Direction','Direction (Guided)']
df1[selected]

,text,Direction,Direction (Guided)
0,Question:\n\nSell 1 naked GME call here.\n\nWorth it?\n\nDangerous?\n\nOnly 1 shouldn’t be too bad?,Short,"Taking profit suggests that the user is closing a long position, therefore the tweet indicates closing a long position."
1,$PARA Open\n\nSee how Sony plays the buyout process.\n\nSmall. https://t.co/CceV23s8gb,N/A,"N/A, the tweet does not provide enough information to determine whether the poster is opening a long or short position."
2,Sold my $INTC calls too soon. But shares doing well.\n\nAnyone hopped in the INTC trade?,N/A,"N/A, as the tweet does not provide enough information to determine whether the poster is opening a long or short position."
3,$VKTX Open https://t.co/2G1TFQ8iaV,Long,"N/A, the tweet does not provide enough information to determine whether it is indicative of opening a long or short position."
4,"$INTC Closed, switched to shares\n\nIn 1.15 (May 8th)\n\nOut 1.29\n\n+12.1% \n+$140\n\nSwitched to 400 shares instead. Still looking for 32-33’s. https://t.co/LeXt1fPshM",Short,This tweet is N/A as it does not provide enough information to determine whether it suggests opening a long or short position.


In [62]:
display(heisenberg_tweets.head(5))

,id,text,created_at,image_url,Image
0,1790077422697693598,Question:\n\nSell 1 naked GME call here.\n\nWorth it?\n\nDangerous?\n\nOnly 1 shouldn’t be too bad?,2024-05-13 17:51:30+00:00,0,No image available
1,1790077076105498901,$PARA Open\n\nSee how Sony plays the buyout process.\n\nSmall. https://t.co/CceV23s8gb,2024-05-13 17:50:07+00:00,[pic.twitter.com/CceV23s8gb],None
2,1790026595027005823,Sold my $INTC calls too soon. But shares doing well.\n\nAnyone hopped in the INTC trade?,2024-05-13 14:29:31+00:00,0,No image available
3,1790014939173630180,$VKTX Open https://t.co/2G1TFQ8iaV,2024-05-13 13:43:13+00:00,[pic.twitter.com/2G1TFQ8iaV],None
4,1790014760244621631,"$INTC Closed, switched to shares\n\nIn 1.15 (May 8th)\n\nOut 1.29\n\n+12.1% \n+$140\n\nSwitched to 400 shares instead. Still looking for 32-33’s. https://t.co/LeXt1fPshM",2024-05-13 13:42:30+00:00,"[pic.twitter.com/LeXt1fPshM, pic.twitter.com/LeXt1fPshM]",None


In [67]:
def initialize_webdriver():
    options = Options()
    options.add_argument("--headless")
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    service = Service(ChromeDriverManager().install())
    return webdriver.Chrome(service=service, options=options)

def get_jpg_url(driver, url):
    try:
        # Check if URL is a list and extract the first element
        if isinstance(url, list):
            if len(url) > 0:
                url = url[0]
            else:
                return None
        
        # Convert URL to string if it is not None
        if url is None or url == 0:
            return None
        if not isinstance(url, str):
            url = str(url)
        
        if not url.startswith('http'):
            url = 'https://' + url

        driver.get(url)
        time.sleep(5)  # Allow time for the page to load

        redirected_url = driver.current_url
        if redirected_url != url:
            driver.get(redirected_url)
            time.sleep(5)

        images = driver.find_elements("tag name", "img")
        for img in images:
            img_src = img.get_attribute('src')
            if 'media' in img_src:
                return img_src
    except Exception as e:
        print(f"Error processing URL {url}: {e}")
    return None

driver = initialize_webdriver()

try:
    heisenberg_tweets['jpg_url'] = heisenberg_tweets['image_url'].apply(lambda x: get_jpg_url(driver, x))
finally:
    driver.quit()

In [71]:
heisenberg_tweets['image_response'] = heisenberg_tweets['jpg_url'].apply(lambda x: get_response_image(x) if x != None else None)
display(heisenberg_tweets)

,id,text,created_at,image_url,Image,jpg_url,image_response
0,1790077422697693598,Question:\n\nSell 1 naked GME call here.\n\nWorth it?\n\nDangerous?\n\nOnly 1 shouldn’t be too bad?,2024-05-13 17:51:30+00:00,0,No image available,None,None
1,1790077076105498901,$PARA Open\n\nSee how Sony plays the buyout process.\n\nSmall. https://t.co/CceV23s8gb,2024-05-13 17:50:07+00:00,[pic.twitter.com/CceV23s8gb],None,https://pbs.twimg.com/media/GNehnPvbMAAq-N7?format=jpg&name=small,"The image depicts a stock trade confirmation for ""PARA,"" specifically showing the purchase of 500 shares at a limit price of 13.11, with an average fill price of 13.1099. This does not appear to describe an options play but rather a straightforward stock purchase. \n\nThe key details are:\n- Ticker: PARA\n- Quantity: +500 shares\n- Purchase Price: 13.1099 per share\n- Type of Order: Limit Order (LMT) at 13.11\n\nThis indicates the trader placed a limit order to buy 500 shares of PARA stock at a price of 13.11 or better, and it was filled at an average price of 13.1099."
2,1790026595027005823,Sold my $INTC calls too soon. But shares doing well.\n\nAnyone hopped in the INTC trade?,2024-05-13 14:29:31+00:00,0,No image available,None,None
3,1790014939173630180,$VKTX Open https://t.co/2G1TFQ8iaV,2024-05-13 13:43:13+00:00,[pic.twitter.com/2G1TFQ8iaV],None,https://pbs.twimg.com/media/GNdpGbAbwAA4VSw?format=jpg&name=small,"The image displays a stock buying transaction, not an option play. The details given are:\n\n- Ticker symbol: VKTX\n- A total of 100 shares were bought at an average fill price of $73.93\n- Two trade fills occurred at 6:40 AM on May 13, 2024:\n - 1 share was bought at $73.93\n - 99 shares were bought at $73.93\n\nThe order type is listed as a limit order (LMT) at $73.93."
4,1790014760244621631,"$INTC Closed, switched to shares\n\nIn 1.15 (May 8th)\n\nOut 1.29\n\n+12.1% \n+$140\n\nSwitched to 400 shares instead. Still looking for 32-33’s. https://t.co/LeXt1fPshM",2024-05-13 13:42:30+00:00,"[pic.twitter.com/LeXt1fPshM, pic.twitter.com/LeXt1fPshM]",None,https://pbs.twimg.com/media/GNdo78CaEAAUJDX?format=jpg&name=small,"The image depicts a covered call option trade. Specifically, it shows the sale of 10 call contracts (with each contract representing 100 shares) for Intel Corporation (INTC) with a strike price of $30, expiring on June 7, 2024. The contracts were sold at a price of $1.29 per share.\n\nIn summary:\n- **Ticker:** INTC (Intel Corporation)\n- **Type:** Call option\n- **Expiration Date:** June 7, 2024\n- **Strike Price:** $30\n- **Quantity:** 10 contracts\n- **Action:** Sell\n- **Price per Share:** $1.29\n\nThe seller of the call option receives a premium of $1.29 per share (or $129 per contract) and is obligated to sell the underlying stock at the strike price of $30 if the option is exercised by the buyer."
